# Import Libraries

In [90]:
import matplotlib.pyplot as plt
import pandas as pd
import spacy


In [91]:
company_data = pd.read_csv('../data/input/ml_insurance_challenge.csv')
taxonomy_data = pd.read_csv('../data/input/insurance_taxonomy.csv')

In [92]:
company_data.isnull().sum()      # Missing Data

description      12
business_tags     0
sector           27
category         27
niche             0
dtype: int64

We have missing data, we have two options:

(a) Remove rows with missing values

(b) Fill missing data with a default value

I delete the missing data because i don't want to fill in with wrong information (being a small number of data we don't lose a large amount of information)

In [93]:
company_data.dropna(inplace=True)
company_data.isnull().sum()

description      0
business_tags    0
sector           0
category         0
niche            0
dtype: int64

Eliminate noise (special characters, stopwords & apply Lemmatizer( "running", "ran", "runs" -> "run")) that does not bring significant information for classification.

In [94]:
# # Download Resource:
# # nltk.download('stopwords')
# # nltk.download('wordnet')
# # nltk.download('averaged_perceptron_tagger_eng')
#
#
# stop_words = set(stopwords.words('english'))
# lemmatizer = WordNetLemmatizer()
#
#
# def get_wordnet_pos(word):
#     tag = pos_tag([word])[0][1][0].upper()
#     tag_dict = {
#         "J": wordnet.ADJ,
#         "N": wordnet.NOUN,
#         "V": wordnet.VERB,
#         "R": wordnet.ADV
#     }
#
#     if word.endswith("ing"):
#         return wordnet.VERB
#
#     return tag_dict.get(tag, wordnet.NOUN)
#
#
# def lemmatize_text(text):
#     words = text.split()
#     return ' '.join(
#         [lemmatizer.lemmatize(word=word, pos=get_wordnet_pos(word)) for word in words]
#     )
#
#
# def clean_text(text):
#     if isinstance(text, str):
#         text = text.lower()
#         text = re.sub(r'[^a-z\s]', '', text)
#         words = text.split()
#         words = [word for word in words if word not in stop_words]
#         text = ' '.join(words)
#         lemmatized_text = lemmatize_text(text)
#         return lemmatized_text
#
#     return ''

In [95]:
# company_data['clean_description'] = company_data['description'].apply(clean_text)

In [96]:
# !python -m spacy download en_core_web_sm

nlp = spacy.load("en_core_web_sm")

def lemmatize_text_spacy(text):
    doc = nlp(text)
    lemmatized_words = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]

    return " ".join(lemmatized_words)


company_data['clean_description'] = company_data['description'].apply(lemmatize_text_spacy)

In [99]:
company_data.iloc[0, 5]

'welchcivil civil engineering construction company specialize design build utility network connection UK offer multi utility solution combine electricity gas water fibre optic installation single contract design engineer team capable design electricity water gas network exist network connection point meter location development project management reinforcement diversion provide custom connection solution account exist asset maximize usage trench meet project deadline welchcivil considerable expertise instal gas electricity connection variety market category include residential commercial industrial project'